## 1 定义图像转换函数

In [2]:
# 下载数据
!wget -P /root/jupyter_notebook http://datasrc.tipdm.net:81/python/case/RectalCancer/10001.dcm
!wget -P /root/jupyter_notebook http://datasrc.tipdm.net:81/python/case/RectalCancer/10001_mask.png

--2022-03-30 14:37:40--  http://datasrc.tipdm.net:81/python/case/RectalCancer/10001.dcm
正在解析主机 datasrc.tipdm.net... 203.88.218.216
正在连接 datasrc.tipdm.net|203.88.218.216|:81... 已连接。
已发出 HTTP 请求，正在等待回应... 200 OK
长度：180316 (176K)
正在保存至: “/root/jupyter_notebook/10001.dcm”

100%[======================================>] 180,316     --.-K/s   in 0.002s  

2022-03-30 14:37:40 (94.3 MB/s) - 已保存 “/root/jupyter_notebook/10001.dcm” [180316/180316])

--2022-03-30 14:37:40--  http://datasrc.tipdm.net:81/python/case/RectalCancer/10001_mask.png
正在解析主机 datasrc.tipdm.net... 203.88.218.216
正在连接 datasrc.tipdm.net|203.88.218.216|:81... 已连接。
已发出 HTTP 请求，正在等待回应... 200 OK
长度：334 [image/png]
正在保存至: “/root/jupyter_notebook/10001_mask.png”

100%[======================================>] 334         --.-K/s   in 0s      

2022-03-30 14:37:40 (48.4 MB/s) - 已保存 “/root/jupyter_notebook/10001_mask.png” [334/334])



In [2]:
# 导入库
import SimpleITK as sitk
import radiomics
from radiomics.firstorder import RadiomicsFirstOrder
from radiomics.shape import RadiomicsShape
from radiomics.shape2D import RadiomicsShape2D
from radiomics.glcm import RadiomicsGLCM
from radiomics.glszm import RadiomicsGLSZM
from radiomics.glrlm import RadiomicsGLRLM
from radiomics.ngtdm import RadiomicsNGTDM
from radiomics.gldm import RadiomicsGLDM

def change(inputImage_path,inputMask_path):
    """
    dcm图为三维
    inputMask = sitk.ReadImage(inputMask_path)
    这样读取出来的mask图为二维的
    """
    inputImage_path=tuple([inputImage_path])
    inputImage = sitk.ReadImage(inputImage_path)
    """
    将mask二维转换为三维
    """
    inputMask_path = tuple([inputMask_path])
    reader = sitk.ImageSeriesReader()
    reader.SetFileNames(inputMask_path)
    inputMask = reader.Execute()
    return inputImage,inputMask

ModuleNotFoundError: No module named 'radiomics'

## 2 定义一阶特征函数

In [3]:
def first_order_features(inputImage,inputMask):
    """
    一阶特征
    :param inputImage:
    :param inputMask:
    :param kwargs:
    :return:
    """
    radiomics = RadiomicsFirstOrder(inputImage,inputMask)
    #调用计算纹理矩阵功能类
    radiomics._initCalculation()
    #输入各一阶特征的值
    enerage=radiomics.getEnergyFeatureValue()[0]
    #print("能量值：",enerage)
    allenerage=radiomics.getTotalEnergyFeatureValue()[0]
    #print("总能量:",allenerage)
    entropy=radiomics.getEntropyFeatureValue()[0]
    #print("熵：",entropy)
    minimum=radiomics.getMinimumFeatureValue()[0]
    #print("最低限度:",minimum)
    tenmean=radiomics.get10PercentileFeatureValue()[0]
    #print("第十个百分位数：",tenmean)
    ninetymean=radiomics.get90PercentileFeatureValue()[0]
    #print("第九十个百分位数:",ninetymean)
    maxnum=radiomics.getMaximumFeatureValue()[0]
    #print("最大限度:",maxnum)
    average_gray_level=radiomics.getMeanFeatureValue()[0]
    #print("平均灰度强度:",average_gray_level)
    median_gray_level=radiomics.getMedianFeatureValue()[0]
    #print("中位灰度强度:",median_gray_level)
    InterquartileRange=radiomics.getInterquartileRangeFeatureValue()[0]
    #print("四分位数范围:",InterquartileRange)
    Range=radiomics.getRangeFeatureValue()[0]
    #print("范围：",Range)
    MADs=radiomics.getMeanAbsoluteDeviationFeatureValue()[0]
    #print("平均绝对偏差（MAD)",MADs)
    rMAD=radiomics.getRobustMeanAbsoluteDeviationFeatureValue()[0]
    #print("稳健的平均绝对偏差（rMAD）",rMAD)
    RootMeanSquared=radiomics.getRootMeanSquaredFeatureValue()[0]
    #print("均平方根：",RootMeanSquared)
    StandardDeviation=radiomics.getStandardDeviationFeatureValue()[0]
    #print("标准偏差:",StandardDeviation)
    Skewness=radiomics.getSkewnessFeatureValue()[0]
    #print("歪斜:",Skewness)
    Kurtosis=radiomics.getKurtosisFeatureValue()[0]
    #print("Kurtosis:",Kurtosis)
    Variance=radiomics.getVarianceFeatureValue()[0]
    #print("差异:",Variance)
    Uniformity=radiomics.getUniformityFeatureValue()[0]
    #print("一致性:",Uniformity)
    #将所有特征存入数组
    firstlist=[enerage,allenerage,entropy,minimum,
    tenmean,ninetymean,maxnum,average_gray_level,
    median_gray_level,InterquartileRange,Range,
    MADs,rMAD,RootMeanSquared,StandardDeviation,
    Skewness,Kurtosis,Variance,Uniformity]
    return firstlist

## 3 定义形状特征函数

In [4]:
def shape_3D(inputImage,inputMask,**kwargs):
    """
    形状特征（3D）
    :param inputImage:
    :param inputMask:
    :param kwargs:
    :return:
    """
    radiomics = RadiomicsShape(inputImage,inputMask)
    MeshVolume=radiomics.getMeshVolumeFeatureValue()
    #print("网格体积:",MeshVolume)
    VoxelVolume=radiomics.getVoxelVolumeFeatureValue()
    #print("体素量:",VoxelVolume)
    SurfaceArea=radiomics.getSurfaceAreaFeatureValue()
    #print("表面积:",SurfaceArea)
    SurfaceVolume=radiomics.getSurfaceVolumeRatioFeatureValue()
    #print("表面积与体积比:",SurfaceVolume)
    Sphericity=radiomics.getSphericityFeatureValue()
    #print("球形:",Sphericity)
    Compactness1=radiomics.getCompactness1FeatureValue()
    #print("紧凑性1:",Compactness1)
    Compactness2=radiomics.getCompactness2FeatureValue()
    #print("紧凑性2:",Compactness2)
    SphericalDisproportion=radiomics.getSphericalDisproportionFeatureValue()
    #print("球形比例:",SphericalDisproportion)
    Maximum3DDiameter=radiomics.getMaximum3DDiameterFeatureValue()
    #print("最大3D直径:",Maximum3DDiameter)
    Maximum2DDiameterSlice=radiomics.getMaximum2DDiameterSliceFeatureValue()
    #print("最大2D直径（切片):",Maximum2DDiameterSlice)
    Maximum2DDiameterColumn=radiomics.getMaximum2DDiameterColumnFeatureValue()
    #print("最大2D直径（列):",Maximum2DDiameterColumn)
    Maximum2DDiameterRow=radiomics.getMaximum2DDiameterRowFeatureValue()
    #print("最大2D直径（行):",Maximum2DDiameterRow)
    MajorAxisLength=radiomics.getMajorAxisLengthFeatureValue()
    #print("主轴长度:",MajorAxisLength)
    MinorAxisLength=radiomics.getMinorAxisLengthFeatureValue()
    #print("小轴长度:",MinorAxisLength)
    LeastAxisLength=radiomics.getLeastAxisLengthFeatureValue()
    #print("最小轴长度:",LeastAxisLength)
    Elongation=radiomics.getElongationFeatureValue()
    #print("伸长率:",Elongation)
    Flatness=radiomics.getFlatnessFeatureValue()
    #print("平整度:",Flatness)
    shap3dlist=[MeshVolume,VoxelVolume,SurfaceArea,
    SurfaceVolume,Sphericity, Compactness1,Compactness2,
    SphericalDisproportion,Maximum3DDiameter,Maximum2DDiameterSlice,
    Maximum2DDiameterColumn,Maximum2DDiameterRow,
    MajorAxisLength,MinorAxisLength,LeastAxisLength,
    Elongation,Flatness]
    return shap3dlist

## 4 定义灰度共生矩阵（GLCM）特征函数

In [5]:
def glcm(inputImage,inputMask,**kwargs):
    """
    灰度共生矩阵（GLCM）特征
    :param inputImage:
    :param inputMask:
    :param kwargs:
    :return:
    """
    radiomics = RadiomicsGLCM(inputImage,inputMask)
    radiomics._initCalculation()
    Autocorrelation=radiomics.getAutocorrelationFeatureValue()[0]
    #print("自相关:",Autocorrelation)
    JointAverage=radiomics.getJointAverageFeatureValue()[0]
    #print("联合平均数:",JointAverage)
    ClusterProminence=radiomics.getClusterProminenceFeatureValue()[0]
    #print("集群突出:",ClusterProminence)
    ClusterShade=radiomics.getClusterShadeFeatureValue()[0]
    #print("群集阴影:",ClusterShade)
    ClusterTendency=radiomics.getClusterTendencyFeatureValue()[0]
    #print("集群趋势:",ClusterTendency)
    Contrast=radiomics.getContrastFeatureValue()[0]
    #print("对比:",Contrast)
    Correlation=radiomics.getCorrelationFeatureValue()[0]
    #print("相关性:",Correlation)
    DifferenceAverage=radiomics.getDifferenceAverageFeatureValue()[0]
    #print("差异平均值:",DifferenceAverage)
    DifferenceEntropy=radiomics.getDifferenceEntropyFeatureValue()[0]
    #print("差异熵:",DifferenceEntropy)
    DifferenceVariance=radiomics.getDifferenceVarianceFeatureValue()[0]
    #print("差异方差:",DifferenceVariance)
    JointEnergy=radiomics.getJointEnergyFeatureValue()[0]
    #print("联合能源:",JointEnergy)
    JointEntropy=radiomics.getJointEntropyFeatureValue()[0]
    #print("联合熵:",JointEntropy)
    Imc1=radiomics.getImc1FeatureValue()[0]
    Imc2=radiomics.getImc2FeatureValue()[0]
    #print("信息相关度量（IMC）1:",Imc1)
    #print("信息相关度量（IMC）2:",Imc2)
    Idm=radiomics.getIdmFeatureValue()[0]
    #print("反差时刻（IDM）:",Idm)
    Mcc=radiomics.getMCCFeatureValue()
    #print("最大相关系数（MCC）:",Mcc)
    Idmn=radiomics.getIdmnFeatureValue()[0]
    #print("逆差分矩归一化（IDMN）:",Idmn)
    Id=radiomics.getIdFeatureValue()[0]
    #print("逆差（ID）:",Id)
    Idn=radiomics.getIdnFeatureValue()[0]
    #print("逆差归一化（IDN）:",Idn)
    InverseVariance=radiomics.getInverseVarianceFeatureValue()[0]
    #print("反方差:",InverseVariance)
    MaximumProbability=radiomics.getMaximumProbabilityFeatureValue()[0]
    #print("最大概率:",MaximumProbability)
    SumEntropy=radiomics.getSumEntropyFeatureValue()[0]
    #print("邻域强度值差异的总和:",SumEntropy)
    SumSquares=radiomics.getSumSquaresFeatureValue()[0]
    #print("平方和:",SumSquares)
    glcmlist=[Autocorrelation,JointAverage,ClusterProminence,
    ClusterShade,ClusterTendency,Contrast,Correlation,
    DifferenceAverage,DifferenceEntropy,DifferenceVariance,
    JointEnergy,JointEntropy,Imc1,Imc2,Idm,Mcc,Idmn,Id,Idn,
    InverseVariance,MaximumProbability,
    SumEntropy,SumSquares]
    return glcmlist

## 5 定义灰度级区域矩阵（GLSZM）功能函数

In [6]:
def glszm(inputImage,inputMask,**kwargs):
    """
    灰度级区域矩阵（GLSZM）功能
    :param inputImage:
    :param intputMask:
    :param kwargs:
    :return:
    """
    radiomics = RadiomicsGLSZM(inputImage,inputMask)
    radiomics._initCalculation()
    SAE=radiomics.getSmallAreaEmphasisFeatureValue()[0]
    #print("小区域重点（SAE):",SAE)
    LAE=radiomics.getLargeAreaEmphasisFeatureValue()[0]
    #print("大面积重点（LAE):",LAE)
    GLN=radiomics.getGrayLevelNonUniformityFeatureValue()[0]
    #print("灰度不均匀性（GLN):",GLN)
    GLNN=radiomics.getGrayLevelNonUniformityNormalizedFeatureValue()[0]
    #print("灰度级非均匀性归一化（GLNN):",GLNN)
    SZN=radiomics.getSizeZoneNonUniformityFeatureValue()[0]
    #print("尺寸区域不均匀性（SZN):",SZN)
    SZNN=radiomics.getSizeZoneNonUniformityNormalizedFeatureValue()[0]
    #print("尺寸区非均匀性归一化（SZNN）:",SZNN)
    ZP=radiomics.getZonePercentageFeatureValue()[0]
    #print("区域百分比（ZP):",ZP)
    GLV=radiomics.getGrayLevelVarianceFeatureValue()[0]
    #print("灰度变化（GLV）:",GLV)
    ZV=radiomics.getZoneVarianceFeatureValue()[0]
    #print("区域差异（ZV）:",ZV)
    ZE=radiomics.getZoneEntropyFeatureValue()[0]
    #print("区域熵（ZE）:",ZE)
    LGLZE=radiomics.getLowGrayLevelZoneEmphasisFeatureValue()[0]
    #print("低灰度区强调（LGLZE）:",LGLZE)
    HGLZE=radiomics.getHighGrayLevelZoneEmphasisFeatureValue()[0]
    #print("高灰度级区域强调（HGLZE）:",HGLZE)
    SALGLE=radiomics.getSmallAreaLowGrayLevelEmphasisFeatureValue()[0]
    #print("小面积低灰度强调（SALGLE):",SALGLE)
    SAHGLE=radiomics.getSmallAreaHighGrayLevelEmphasisFeatureValue()[0]
    #print("小区域高灰度重点（SAHGLE）:",SAHGLE)
    LALGLE=radiomics.getLargeAreaLowGrayLevelEmphasisFeatureValue()[0]
    #print("大面积低灰度强调（LALGLE）:",LALGLE)
    LAHGLE=radiomics.getLargeAreaHighGrayLevelEmphasisFeatureValue()[0]
    #print("大面积高灰度强调（LAHGLE):",LAHGLE)
    glszelist=[SAE,LAE,GLN,GLNN,SZN,SZNN,ZP,GLV,
    ZV,ZE,LGLZE,HGLZE,SALGLE,SAHGLE,LALGLE,LAHGLE]
    return glszelist

## 6 定义灰度级别运行长度矩阵（GLRLM）功能函数

In [7]:
def glrlm(inputImage,inputMask,**kwargs):
    """
    灰度级别运行长度矩阵（GLRLM）功能
    :param inputImage:
    :param inputMask:
    :param kwargs:
    :return:
    """
    radiomics = RadiomicsGLRLM(inputImage,inputMask)
    radiomics._initCalculation()
    SRE=radiomics.getShortRunEmphasisFeatureValue()[0]
    #print("短期强调（SRE）:",SRE)
    LRE=radiomics.getLongRunEmphasisFeatureValue()[0]
    #print("长期强调（LRE）:",LRE)
    GLN=radiomics.getGrayLevelNonUniformityFeatureValue()[0]
    #print("灰度不均匀性（GLN):",GLN)
    GLNN=radiomics.getGrayLevelNonUniformityNormalizedFeatureValue()[0]
    #print("灰度级非均匀性归一化（GLNN）:",GLNN)
    RLN=radiomics.getRunLengthNonUniformityFeatureValue()[0]
    #print("运行长度不均匀性（RLN):",RLN)
    RLNN=radiomics.getRunLengthNonUniformityNormalizedFeatureValue()[0]
    #print("运行长度非均匀性归一化（RLNN）:",RLNN)
    RP=radiomics.getRunPercentageFeatureValue()[0]
    #print("运行百分比（RP）:",RP)
    GLV=radiomics.getGrayLevelVarianceFeatureValue()[0]
    #print("灰度变化（GLV）:",GLV)
    RV=radiomics.getRunVarianceFeatureValue()[0]
    #print("运行方差（RV）:",RV)
    RE=radiomics.getRunEntropyFeatureValue()[0]
    #print("运行熵（RE）:",RE)
    LGLRE=radiomics.getLowGrayLevelRunEmphasisFeatureValue()[0]
    #print("低灰度级运行强调（LGLRE）:",LGLRE)
    HGLRE=radiomics.getHighGrayLevelRunEmphasisFeatureValue()[0]
    #print("高灰度级运行强调（HGLRE）:",HGLRE)
    SRLGLE=radiomics.getShortRunLowGrayLevelEmphasisFeatureValue()[0]
    #print("短期低灰度级强调（SRLGLE）:",SRLGLE)
    SRHGLE=radiomics.getShortRunHighGrayLevelEmphasisFeatureValue()[0]
    #print("短期高灰度强调（SRHGLE）:",SRHGLE)
    LRLGLE=radiomics.getLongRunLowGrayLevelEmphasisFeatureValue()[0]
    #print("长期低灰度级强调（LRLGLE）:",LRLGLE)
    LRHGLE=radiomics.getLongRunHighGrayLevelEmphasisFeatureValue()[0]
    #print("长期高灰度强调（LRHGLE）:",LRHGLE)
    glrlmlist=[SRE,LRE,GLN,GLNN,RLN,RLNN,RP,GLV,RV,RE,
    LGLRE,HGLRE,SRLGLE,SRHGLE,LRLGLE,LRHGLE]
    return glrlmlist

## 7 定义相邻灰度差矩阵（NGTDM）特征函数

In [8]:
def ngtdm(inputImage,inputMask,**kwargs):
    """
    相邻灰度差矩阵（NGTDM）特征
    :param inputImage:
    :param inputMask:
    :param kwargs:
    :return:
    """
    radiomics = RadiomicsNGTDM(inputImage,inputMask)
    radiomics._initCalculation()
    Coarseness=radiomics.getCoarsenessFeatureValue()[0]
    #print("粗糙度:",Coarseness)
    Contrast=radiomics.getContrastFeatureValue()[0]
    #print("对比度:",Contrast)
    Busyness=radiomics.getBusynessFeatureValue()[0]
    #print("繁忙程度:",Busyness)
    Complexity=radiomics.getComplexityFeatureValue()[0]
    #print("复杂性:",Complexity)
    Strength=radiomics.getStrengthFeatureValue()[0]
    #print("强度:",Strength)
    ngtdmlist=[Coarseness,Contrast,Busyness,Complexity,Strength]
    return ngtdmlist

## 8 定义灰度级依赖矩阵（GLDM）特征函数

In [9]:
def gldm(inputImage,inputMask,**kwargs):
    """
    灰度级依赖矩阵（GLDM）特征
    :param inputImage:
    :param inputMask:
    :param kwargs:
    :return:
    """
    radiomics = RadiomicsGLDM(inputImage,inputMask)
    radiomics._initCalculation()
    SDE=radiomics.getSmallDependenceEmphasisFeatureValue()[0]
    #print("小依赖性重点（SDE):",SDE)
    LDE=radiomics.getLargeDependenceEmphasisFeatureValue()[0]
    #print("大依赖性重点（LDE）:",LDE)
    GLN=radiomics.getGrayLevelNonUniformityFeatureValue()[0]
    #print("灰度不均匀性（GLN):",GLN)
    DN=radiomics.getDependenceNonUniformityFeatureValue()[0]
    #print("依赖性不均匀性（DN）:",DN)
    DNN=radiomics.getDependenceNonUniformityNormalizedFeatureValue()[0]
    #print("依赖性非均匀性归一化（DNN):",DNN)
    GLV=radiomics.getGrayLevelVarianceFeatureValue()[0]
    #print("灰度变化（GLV):",GLV)
    DV=radiomics.getDependenceVarianceFeatureValue()[0]
    #print("依赖方差（DV):",DV)
    DE=radiomics.getDependenceEntropyFeatureValue()[0]
    #print("依赖熵（DE）:",DE)
    LGLE=radiomics.getLowGrayLevelEmphasisFeatureValue()[0]
    #print("低灰度强调（LGLE）:",LGLE)
    HGLE=radiomics.getHighGrayLevelEmphasisFeatureValue()[0]
    #print("高灰度强调（HGLE):",HGLE)
    SDLGLE=radiomics.getSmallDependenceLowGrayLevelEmphasisFeatureValue()[0]
    #print("小依赖性低灰度级强调（SDLGLE):",SDLGLE)
    SDHGLE=radiomics.getSmallDependenceHighGrayLevelEmphasisFeatureValue()[0]
    #print("小依赖性高灰度级强调（SDHGLE):",SDHGLE)
    LDLGLE=radiomics.getLargeDependenceLowGrayLevelEmphasisFeatureValue()[0]
    #print("大依赖性低灰度强调（LDLGLE):",LDLGLE)
    LDHGLE=radiomics.getLargeDependenceHighGrayLevelEmphasisFeatureValue()[0]
    #print("大依赖性高灰度级强调（LDHGLE）:",LDHGLE)
    gldmlist=[SDE,LDE,GLN,DN,DNN,GLV,DV,DE,LGLE,HGLE,
    SDLGLE,SDHGLE,LDLGLE,LDHGLE]
    return gldmlist

## 9 实现图像转换与特征提取

In [10]:
def features(inputImage_path,inputMask_path):
    inputImage,inputMask = change(inputImage_path,inputMask_path)
    a=first_order_features(inputImage,inputMask)
    b=shape_3D(inputImage,inputMask)
    c=glcm(inputImage,inputMask)
    d=ngtdm(inputImage,inputMask)
    e=gldm(inputImage,inputMask)
    f=glrlm(inputImage,inputMask)
    g=glszm(inputImage,inputMask)
    features=a+b+c+d+e+f+g
    return features,len(features)

inputMask_path=r'/root/jupyter_notebook/10001.dcm'
inputImage_path=r'/root/jupyter_notebook/10001_mask.png'
features(inputImage_path,inputMask_path)

([0.0,
  0.0,
  -3.2034265038149176e-16,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  50.666666666666664,
  284.0,
  506.33029413649444,
  9.993361068483443,
  0.13077711958352498,
  0.0025089761232965305,
  0.002236635958989141,
  7.646597533151188,
  410.8004381691918,
  410.8004381691918,
  387.0,
  256.0,
  394.1108988814788,
  291.72048913092493,
  0.0,
  0.740198989570837,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  -3.2034265038149176e-16,
  0.0,
  1.0,
  -3.2034265038149176e-16,
  0.0,
  0.0,
  1.0,
  1,
  1.0,
  1.0,
  1.0,
  0.0,
  1.0,
  -3.2034265038149176e-16,
  0.0,
  1000000.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.9166666666666666,
  1.380281690140845,
  284.0,
  228.17605633802816,
  0.8034368180916485,
  0.0,
  0.12651259670700257,
  0.5474295148658189,
  1.0,
  1.0,
  0.9166666666666666,
  0.9166666666666666,
  1.380281690140845,
  1.380281690140845,
  0.9891438755452171,
  1.04742130